<a href="https://colab.research.google.com/github/MaliheDahmardeh/Olympic-History/blob/main/Olympic-History.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [20]:
import numpy as np
import pandas as pd
import os
import datetime

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC ,SVR
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split 
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from sklearn.decomposition import PCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from pandas.core.algorithms import mode

In [21]:
!ls

athlete_events.csv  sample_data


In [22]:
df = pd.read_csv('athlete_events.csv')

In [23]:
df

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135569,Andrzej ya,M,29.0,179.0,89.0,Poland-1,POL,1976 Winter,1976,Winter,Innsbruck,Luge,Luge Mixed (Men)'s Doubles,NaN
271112,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Individual",NaN
271113,135570,Piotr ya,M,27.0,176.0,59.0,Poland,POL,2014 Winter,2014,Winter,Sochi,Ski Jumping,"Ski Jumping Men's Large Hill, Team",NaN
271114,135571,Tomasz Ireneusz ya,M,30.0,185.0,96.0,Poland,POL,1998 Winter,1998,Winter,Nagano,Bobsleigh,Bobsleigh Men's Four,NaN


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271116 entries, 0 to 271115
Data columns (total 15 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   ID      271116 non-null  int64  
 1   Name    271116 non-null  object 
 2   Sex     271116 non-null  object 
 3   Age     261642 non-null  float64
 4   Height  210945 non-null  float64
 5   Weight  208241 non-null  float64
 6   Team    271116 non-null  object 
 7   NOC     271116 non-null  object 
 8   Games   271116 non-null  object 
 9   Year    271116 non-null  int64  
 10  Season  271116 non-null  object 
 11  City    271116 non-null  object 
 12  Sport   271116 non-null  object 
 13  Event   271116 non-null  object 
 14  Medal   39783 non-null   object 
dtypes: float64(3), int64(2), object(10)
memory usage: 31.0+ MB


In [25]:
df.shape

(271116, 15)

#Data Cleaning

In [26]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age         9474
Height     60171
Weight     62875
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     231333
dtype: int64

In [27]:
df.duplicated().sum()

1385

In [28]:
df.drop_duplicates(inplace=True)

In [29]:
df.duplicated().sum()

0

Fill  null value for Age, Height and Weight with mean of them

Mean of height


In [30]:
mean_height=df["Height"].mean()
mean_height

175.33895323752947

In [31]:
df["Height"]=df["Height"].fillna(mean_height)
df["Height"]

0         180.000000
1         170.000000
2         175.338953
3         175.338953
4         185.000000
             ...    
271111    179.000000
271112    176.000000
271113    176.000000
271114    185.000000
271115    185.000000
Name: Height, Length: 269731, dtype: float64

Mean of weight

In [32]:
mean_weight=df["Weight"].mean()
mean_weight

70.70177806382202

In [33]:
df["Weight"]=df["Weight"].fillna(mean_weight)
df["Weight"]

0         80.000000
1         60.000000
2         70.701778
3         70.701778
4         82.000000
            ...    
271111    89.000000
271112    59.000000
271113    59.000000
271114    96.000000
271115    96.000000
Name: Weight, Length: 269731, dtype: float64

Mean of Age

In [34]:
mean_age=df["Age"].mean()
mean_age

25.454776204227084

In [35]:
df["Age"]=df["Age"].fillna(mean_age)
df["Age"]

0         24.0
1         23.0
2         24.0
3         34.0
4         21.0
          ... 
271111    29.0
271112    27.0
271113    27.0
271114    30.0
271115    34.0
Name: Age, Length: 269731, dtype: float64

In [36]:
df.isnull().sum()

ID             0
Name           0
Sex            0
Age            0
Height         0
Weight         0
Team           0
NOC            0
Games          0
Year           0
Season         0
City           0
Sport          0
Event          0
Medal     229959
dtype: int64

In [37]:
df.describe

<bound method NDFrame.describe of             ID                      Name Sex   Age      Height     Weight  \
0            1                 A Dijiang   M  24.0  180.000000  80.000000   
1            2                  A Lamusi   M  23.0  170.000000  60.000000   
2            3       Gunnar Nielsen Aaby   M  24.0  175.338953  70.701778   
3            4      Edgar Lindenau Aabye   M  34.0  175.338953  70.701778   
4            5  Christine Jacoba Aaftink   F  21.0  185.000000  82.000000   
...        ...                       ...  ..   ...         ...        ...   
271111  135569                Andrzej ya   M  29.0  179.000000  89.000000   
271112  135570                  Piotr ya   M  27.0  176.000000  59.000000   
271113  135570                  Piotr ya   M  27.0  176.000000  59.000000   
271114  135571        Tomasz Ireneusz ya   M  30.0  185.000000  96.000000   
271115  135571        Tomasz Ireneusz ya   M  34.0  185.000000  96.000000   

                  Team  NOC        Games 